In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, make_scorer, balanced_accuracy_score
import numpy as np
import random
from sklearn.svm import SVC
from sklearn import metrics

In [3]:
train = pd.read_csv('data/svm_OSN_50SPCs_3000G.csv',index_col=0 )
#train = train.reset_index(drop = True)
train.head()

,SPC_1,SPC_2,SPC_3,SPC_4,SPC_5,SPC_6,SPC_7,SPC_8,SPC_9,SPC_10,...,SPC_42,SPC_43,SPC_44,SPC_45,SPC_46,SPC_47,SPC_48,SPC_49,SPC_50,observed
V3F1_CAACGGCAGTAAACAC,-2.436386,-5.286533,-2.847332,2.566147,5.935313,-2.032978,0.304217,0.270347,-0.644762,-1.457629,...,1.314989,-2.082101,0.862903,0.244200,0.116634,-1.002064,0.234064,-0.620904,-0.339079,Olfr477
V3F3_GAGACCCTCTGAGCAT,0.230544,-9.144450,3.919632,2.055721,4.186333,-0.642285,-0.167196,-1.993198,-0.309246,-0.883709,...,-0.966048,0.431340,-0.427532,0.625230,1.988114,-1.800207,0.905261,0.758330,1.099580,Olfr354
V3F1_GAGGGATGTAGCCCTG,-0.442758,-4.558211,-11.266653,0.069706,2.658198,-0.948847,-2.732315,-1.923870,-2.024378,-4.713357,...,0.544582,-0.023299,-0.064075,-0.528713,1.274820,-1.358156,0.575562,0.256719,-0.858667,Olfr711
V3F3_ACGTAGTCAGGACAGT,3.065503,-8.600753,-2.948539,5.526509,-2.190336,-1.581605,-4.218123,3.054864,-1.292138,-5.544259,...,-1.130459,-0.660782,-2.084247,-0.833999,-0.119654,-0.726499,-0.229057,-0.556224,1.687474,Olfr3
V3M1_ACCAAACCAAGTGACG,-4.673190,2.219611,-4.118749,-0.544320,-8.157987,5.068371,1.010966,7.572142,-8.647740,-2.171079,...,1.419901,0.652934,0.434828,-1.140898,0.403794,-0.017617,0.654036,-2.253383,1.210954,Olfr1080


In [4]:
le = LabelEncoder()
X = train.values[:,:50]
y = train["observed"].values
y = le.fit_transform(y)

In [5]:
model = SVC(C = 0.01,kernel = "linear",class_weight =  "balanced", decision_function_shape = "ovo")
model.fit(X, y)

SVC(C=0.01, class_weight='balanced', decision_function_shape='ovo',
    kernel='linear')

In [6]:
test = pd.read_csv('data/svm_iOSN_50SPCs_3000G.csv',index_col=0)
#test = test.reset_index(drop = True)

In [7]:
y_pred = model.predict(test.values[:,:50])

In [8]:
balanced_accuracy_score(le.transform(test["observed"].values), y_pred)

/Users/plg/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


0.07081089374217203

In [9]:
d = {'bc':test.index.values.tolist(),'Ob':test["observed"].values,'Pd':le.inverse_transform(y_pred)}
df = pd.DataFrame(d)

In [10]:
df.to_csv("data/iOSN_prediction_result.csv")

In [12]:
###shuffle the train and predict, should be ~ 1/654
ys = le.transform(train["observed"].values)
np.random.shuffle(ys)

model = SVC(C = 0.01,kernel = "linear",class_weight =  "balanced", decision_function_shape = "ovo")
model.fit(X, ys)
test = pd.read_csv('data/svm_iOSN_50SPCs_3000G.csv',index_col=0) #Use the OR removed 654 OR OSN data with top 3000 variable genes to generate top 50 SPC
test = train.reset_index(drop = True)

In [13]:
y_pred = model.predict(test.values[:,:50])
balanced_accuracy_score(le.transform(test["observed"].values), y_pred)

0.001412546393267042